In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/05 14:39:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, inferSchema=True)




23/06/05 14:39:54 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
df.show(10)

+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|               date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08 00:00:00|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13 00:00:00|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12 00:00:00|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16 00:00:00|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08 00:00:00|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa0052

In [16]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')





In [18]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("SELECT AVG(price) FROM home_sales WHERE bedrooms == 4 GROUP BY year ORDER BY year").show()

# from pyspark.sql import functions as F

# average_prices = home_sales.filter(home_sales.bedrooms == 4) \
#     .groupBy('year') \
#     .agg(F.round(F.avg('price'), 2).alias('average_price')) \
#     .orderBy('year')

# average_prices.show()




AnalysisException: Column 'year' does not exist. Did you mean one of the following? [home_sales.id, home_sales.date, home_sales.price, home_sales.view, home_sales.floors, home_sales.bedrooms, home_sales.bathrooms, home_sales.sqft_lot, home_sales.waterfront, home_sales.date_built, home_sales.sqft_living]; line 1 pos 63;
'Sort ['year ASC NULLS FIRST], true
+- 'Aggregate ['year], [avg(price#20) AS avg(price)#103]
   +- Filter (bedrooms#21 = 4)
      +- SubqueryAlias home_sales
         +- View (`home_sales`, [id#17,date#18,date_built#19,price#20,bedrooms#21,bathrooms#22,sqft_living#23,sqft_lot#24,floors#25,waterfront#26,view#27])
            +- Relation [id#17,date#18,date_built#19,price#20,bedrooms#21,bathrooms#22,sqft_living#23,sqft_lot#24,floors#25,waterfront#26,view#27] csv


In [29]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query = "SELECT AVG(price) FROM home_sales WHERE bedrooms = 3 AND baths = 3 GROUP BY year ORDER BY year"
spark.sql(query).show()


AnalysisException: Column 'baths' does not exist. Did you mean one of the following? [home_sales.id, home_sales.date, home_sales.view, home_sales.price, home_sales.bathrooms, home_sales.floors, home_sales.bedrooms, home_sales.sqft_lot, home_sales.date_built, home_sales.waterfront, home_sales.sqft_living]; line 1 pos 57;
'Sort ['year ASC NULLS FIRST], true
+- 'Aggregate ['year], [unresolvedalias('AVG('price), None)]
   +- 'Filter ((bedrooms#21 = 3) AND ('baths = 3))
      +- SubqueryAlias home_sales
         +- View (`home_sales`, [id#17,date#18,date_built#19,price#20,bedrooms#21,bathrooms#22,sqft_living#23,sqft_lot#24,floors#25,waterfront#26,view#27])
            +- Relation [id#17,date#18,date_built#19,price#20,bedrooms#21,bathrooms#22,sqft_living#23,sqft_lot#24,floors#25,waterfront#26,view#27] csv


In [28]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query = "SELECT AVG(price) FROM home_sales WHERE beds = 3 AND baths = 3 AND floors = 2 AND sqft >= 2000 GROUP BY year ORDER BY year"
spark.sql(query).show()




AnalysisException: Column 'beds' does not exist. Did you mean one of the following? [home_sales.id, home_sales.date, home_sales.view, home_sales.price, home_sales.bedrooms, home_sales.floors, home_sales.sqft_lot, home_sales.bathrooms, home_sales.date_built, home_sales.waterfront, home_sales.sqft_living]; line 1 pos 40;
'Sort ['year ASC NULLS FIRST], true
+- 'Aggregate ['year], [unresolvedalias('AVG('price), None)]
   +- 'Filter ((('beds = 3) AND ('baths = 3)) AND ((floors#25 = 2) AND ('sqft >= 2000)))
      +- SubqueryAlias home_sales
         +- View (`home_sales`, [id#17,date#18,date_built#19,price#20,bedrooms#21,bathrooms#22,sqft_living#23,sqft_lot#24,floors#25,waterfront#26,view#27])
            +- Relation [id#17,date#18,date_built#19,price#20,bedrooms#21,bathrooms#22,sqft_living#23,sqft_lot#24,floors#25,waterfront#26,view#27] csv


In [19]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
query = "SELECT AVG(price) FROM home_sales WHERE price >= 350000 GROUP BY view ORDER BY view"
spark.sql(query).show()



print("--- %s seconds ---" % (time.time() - start_time))

+------------------+
|        avg(price)|
+------------------+
|403848.51461988303|
| 401044.2513368984|
|397389.24752475246|
| 398867.5964912281|
|         399631.89|
| 401471.8248587571|
| 395655.3789473684|
| 403005.7709497207|
|398592.70658682636|
| 401393.3370786517|
|401868.42523364484|
|399548.11891891895|
| 401501.3243243243|
|         398917.98|
|398570.02923976607|
|404673.29545454547|
| 399586.5311004785|
|398474.49029126216|
| 399332.9090909091|
| 398953.1703296703|
+------------------+
only showing top 20 rows

--- 0.5285050868988037 seconds ---


In [20]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable('home_sales')



In [21]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [22]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
query = "SELECT AVG(price) FROM home_sales WHERE price >= 350000 GROUP BY view ORDER BY view"
spark.sql(query).show()




print("--- %s seconds ---" % (time.time() - start_time))


+------------------+
|        avg(price)|
+------------------+
|403848.51461988303|
| 401044.2513368984|
|397389.24752475246|
| 398867.5964912281|
|         399631.89|
| 401471.8248587571|
| 395655.3789473684|
| 403005.7709497207|
|398592.70658682636|
| 401393.3370786517|
|401868.42523364484|
|399548.11891891895|
| 401501.3243243243|
|         398917.98|
|398570.02923976607|
|404673.29545454547|
| 399586.5311004785|
|398474.49029126216|
| 399332.9090909091|
| 398953.1703296703|
+------------------+
only showing top 20 rows

--- 0.3696582317352295 seconds ---


In [23]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data  and read the formatted parquet data into a DataFrame.

spark.conf.set("spark.sql.shuffle.partitions", "8")
spark.conf.set("spark.default.parallelism", "8")



In [24]:
# 11. Read the formatted parquet data.
parquet_file = "s3a://2u-data-curriculum-team/dataviz-classroom/v1.2/22-big-data/home_sales_revised.parquet"
df = spark.read.parquet(parquet_file)



23/06/05 14:51:27 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://2u-data-curriculum-team/dataviz-classroom/v1.2/22-big-data/home_sales_revised.parquet.
java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:53)
	at org.

Py4JJavaError: An error occurred while calling o57.parquet.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:752)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary

In [ ]:
# 12. Create a temporary table for the parquet data.
df.createOrReplaceTempView('home_sales_revised')



In [25]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()
query = "SELECT AVG(price) FROM home_sales_revised WHERE price >= 350000 GROUP BY view ORDER BY view"
spark.sql(query).show()




print("--- %s seconds ---" % (time.time() - start_time))

AnalysisException: Table or view not found: home_sales_revised; line 1 pos 23;
'Sort ['view ASC NULLS FIRST], true
+- 'Aggregate ['view], [unresolvedalias('AVG('price), None)]
   +- 'Filter ('price >= 350000)
      +- 'UnresolvedRelation [home_sales_revised], [], false


In [26]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable('home_sales')



In [27]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')




False